# Dataset Extraction Example

This notebook is an example to extract a dataset from a digital collection described using marcxml files.
This notebook uses thee descriptive metadata from the Moving Image Archive catalogue, which is Scotland’s national collection of moving images.

### Setting up things

In [1]:
# import the libraries we need
# https://pypi.org/project/pymarc/
import pymarc, re, csv
import pandas as pd
from pymarc import parse_xml_to_array    

### Reading original files

In [77]:
csv_out = csv.writer(open('marc_records.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_out.writerow(['title', 'author', 'place_production', 'date', 'extents', 'credits_note', 'subjects', 'summary', 'detail', 'link'])

records = parse_xml_to_array(open('Moving-Image-Archive/Moving-Image-Archive-dataset-MARC.xml'))

for record in records:
    
    title = author = place_production = date = extents = credits_note = subjects = summary = publisher = link =''
    
    # title
    if record['245'] is not None:
      title = record['245']['a']
      if record['245']['b'] is not None:
        title = title + " " + record['245']['b']
    
    # determine author
    if record['100'] is not None:
      author = record['100']['a']
    elif record['110'] is not None:
      author = record['110']['a']
    elif record['700'] is not None:
      author = record['700']['a']
    elif record['710'] is not None:
      author = record['710']['a']
    
    # place_production
    if record['264'] is not None:
      place_production = record['264']['a']
    
    # date
    for f in record.get_fields('264'):
        dates = f.get_subfields('c')
        if len(dates):
            date = dates[0]
            # cleaning date last .
            if date.endswith('.'): date = date[:-1]
    
    
    # Physical Description - extent
    #if record['300'] is not None:
    #  extent = record['300']['a']
    
    for f in record.get_fields('300'):
        extents = f.get_subfields('a')
        if len(extents):
            extent = extents[0]
            # TODO cleaning
        details = f.get_subfields('b')
        if len(details):
            detail = details[0]
            
    # Creation/production credits note
    if record['508'] is not None:
      credits_note = record['508']['a']
    
    # Summary
    if record['520'] is not None:
      summary = record['520']['a']
    
    # subject
    if record['653'] is not None:
        subjects = '' 
        for f in record.get_fields('653'):
            subjects += f.get_subfields('a')[0] + ' -- '
        subjects = re.sub(' -- $', '', subjects)
    
    
    # link
    if record['856'] is not None:
      link = record['856']['u']
    
    
    csv_out.writerow([title,author,place_production,date,extents,credits_note,subjects,summary,detail,link])

## Reading CSV 

In [78]:
# Load the CSV file from GitHub.
# This puts the data in a Pandas DataFrame
df = pd.read_csv('marc_records.csv')

## Have a peek

In [79]:
# Let's have a look inside...
# Note that both the columns and rows are truncated in this preview
df

,title,author,place_production,date,extents,credits_note,subjects,summary,detail,link
0,(GLASGOW TRAMS AND BOTANIC GARDENS).,"RUSSELL, Stanley Livingstone",[Place of production not identified] :,1950.0,['(2.00 mins) : '],"Director, [filmed by Stanley L. Russell, Thame...","Bus Stations and Depots -- Buses and Coaches, ...","The Botanic Gardens, Glasgow with shots of the...","mute, colour",http://movingimage.nls.uk/film/0001
1,"(LAST DAY OF THE TRAMS, GLASGOW).",NaN,[Place of production not identified] :,1962.0,['(28.00 mins) : '],"Director, [filmed by SAAC].",Transport -- Glasgow -- documentary -- amateur,"Footage of the last trams to run in Glasgow, a...","silent, colour",http://movingimage.nls.uk/film/0002
2,INTO THE MISTS.,NaN,[Place of production not identified] :,1956.0,['(10.04 mins) : '],"Director, [filmed by W.S. Dobson].","Ceremonies -- Emotions, Attitudes and Behaviou...",The story of the last Edinburgh tram. Shots o...,"silent, colour",http://movingimage.nls.uk/film/0004
3,"PASSING OF THE TRAMCAR, the.",NaN,[Place of production not identified] :,1962.0,['(63.36 mins) : '],NaN,Ceremonies -- Transport -- Glasgow,Footage of the last tram to run in Glasgow. Th...,"silent, colour",http://movingimage.nls.uk/film/0005
4,SCOTS OF TOMORROW.,Campbell Harper Productions,[Place of production not identified] :,1959.0,['(13.00 mins) : '],"Producer, Campbell Harper Films Ltd..","Art and Artists, general -- Education -- edu...",Scottish school pupils studying scientific and...,"sound, black and white",http://movingimage.nls.uk/film/0007
5,"British Movietone News No. 1127A, GLASGOW UNIV...",NaN,[Place of production not identified] :,1951.0,['(10.00 mins) : '],"Producer, Movietone News.",Birds -- Ceremonies -- Education -- Glasgow --...,Glasgow University celebrates its Fifth Centen...,"sound, black and white",http://movingimage.nls.uk/film/0008
6,FIFTH CENTENARY OF THE UNIVERSITY OF GLASGOW.,NaN,[Place of production not identified] :,1951.0,['(20.00 mins) : '],"Producer, .","Ceremonies -- Education -- Media, Communicatio...",Celebrations in Glasgow attended by students f...,"sound, black and white",http://movingimage.nls.uk/film/0009
7,ABERDEEN UNIVERSITY QUATER CENTENARY CELEBRATI...,NaN,[Place of production not identified] :,1906.0,['(4.30 mins) : '],"Producer, Scottish Film Council.",Transport,Procession of dignitaries in horse-drawn carri...,"silent, black and white",http://movingimage.nls.uk/film/0010
8,MR. HARRY LAUDER LEAVES FOR AMERICA: His Trium...,NaN,[Place of production not identified] :,1913.0,['(3.16 mins) : '],"Producer, Gaumont News.",Leisure and Recreation -- Music Hall -- Ships ...,Harry Lauder leaves for Liverpool from London'...,"silent, black and white",http://movingimage.nls.uk/film/0011
9,GLASGOW'S YESTERDAYS.,"HART, James",[Place of production not identified] :,1922.0,['(13.02 mins) : '],Credits: arranged by C.A. Oakley assisted by ...,Children and Infants -- Dentistry -- Leisure a...,A selection of amateur films made in the early...,"silent, black and white",http://movingimage.nls.uk/film/0012


## Create some summary data
We can use Pandas to give us a quick overview of the dataset.
### What are the column headings?

In [5]:
# What are the column headings?
# Can you see which ones are missing from the truncated sample above?
df.columns

Index(['title', 'author', 'place_production', 'date', 'extents',
       'credits_note', 'subjects', 'summary', 'publisher'],
      dtype='object')

### How many records are there?

In [6]:
# How many items?
len(df)

5987

## Exploring topics
### Create a list of unique topics and sort them alphabetically

In [7]:
# Get unique values
topics = pd.unique(df['subjects'].str.split('|', expand=True).stack()).tolist()
for topic in sorted(topics, key=str.lower):
    print(topic)

Aberdeen -- Aberdeenshire -- amateur
Aberdeen -- Aberdeenshire -- Argyllshire -- Highlands, the -- amateur
Aberdeen -- advertising
Aberdeen -- amateur
Aberdeen -- amateur -- home movies and videos -- women film makers
Aberdeen -- documentary
Aberdeen -- Edinburgh -- amateur -- promotional
Aberdeen -- local topical
Aberdeen -- Orkney Islands -- documentary -- sponsored -- biographical
Aberdeen -- Stirlingshire -- amateur
Aberdeen -- television news
Aberdeen -- training
Aberdeenshire -- amateur
Aberdeenshire -- amateur -- home movies and videos
Aberdeenshire -- Banff -- amateur
Aberdeenshire -- music -- television news
Aberdeenshire -- newsreel
Aberdeenshire -- television news
advertising
advertising -- promotional -- educational
Agriculture -- Aberdeen -- television documentary -- television news
Agriculture -- Aberdeenshire -- amateur
Agriculture -- Aberdeenshire -- Angus -- Fife -- amateur
Agriculture -- Aberdeenshire -- television news
Agriculture -- Air displays and shows -- Employm

Leisure and Recreation -- Perth -- amateur
Leisure and Recreation -- Perth -- public information
Leisure and Recreation -- Politics
Leisure and Recreation -- Politics -- Edinburgh -- amateur
Leisure and Recreation -- Politics -- Glasgow -- amateur -- propaganda
Leisure and Recreation -- Power Resources -- Transport -- Edinburgh -- newsreel
Leisure and Recreation -- Power Resources -- War -- newsreel
Leisure and Recreation -- Religion -- Dundee -- amateur
Leisure and Recreation -- Religion -- Dundee -- Fife -- amateur
Leisure and Recreation -- Religion -- Fife -- amateur
Leisure and Recreation -- Religion -- Morayshire -- local topical
Leisure and Recreation -- Religion -- Ships and Shipping -- Sporting Activities -- Tourism and Travel -- Argyllshire -- Glasgow -- amateur
Leisure and Recreation -- Renfrewshire -- amateur
Leisure and Recreation -- Renfrewshire -- home movies and videos
Leisure and Recreation -- Renfrewshire -- television arts
Leisure and Recreation -- Restaurants -- amat

### How often is each topic used?

In [9]:
# Splits the topic column and counts frequencies
topic_counts = df['subjects'].str.split('--').apply(lambda x: pd.Series(x).value_counts()).sum().astype('int').sort_values(ascending=False).to_frame().reset_index(level=0)
# Add column names
topic_counts.columns = ['subject', 'count']
# Display with horizontal bars
display(topic_counts.style.bar(subset=['count'], color='#d65f5f').set_properties(subset=['count'], **{'width': '300px'}))

,subject,count
0,amateur,1438
1,Glasgow,688
2,Transport,541
3,Leisure and Recreation,518
4,television news,471
5,Edinburgh,466
6,amateur,417
7,local topical,392
8,"Celebrations, Traditions and Customs",358
9,"Employment, Industry and Industrial Relations",344
